<a href="https://colab.research.google.com/github/dariashcherbakovaaa/MA_DSE_Unimi/blob/MultivariateStatistics/02_AMS_251023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from sklearn.datasets import fetch_openml

census = fetch_openml(data_id=1590, parser = 'auto').frame

census.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [8]:
target_name = "class"

target = census[target_name]
data = census.drop(columns = target_name)

In [9]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42, test_size=0.25
)

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# scaler.fit(data.select_dtypes(exclude='category'))
# scaler.transform(data.select_dtypes(exclude='category'))

scaler.fit_transform(data.select_dtypes(exclude='category'))


# doesn't work since we have categorical

array([[-0.99512893,  0.35167453, -1.19725891, -0.14480353, -0.2171271 ,
        -0.03408696],
       [-0.04694151, -0.94552415, -0.41933527, -0.14480353, -0.2171271 ,
         0.77292975],
       [-0.77631645,  1.3947231 ,  0.74755018, -0.14480353, -0.2171271 ,
        -0.03408696],
       ...,
       [ 1.41180837, -0.35751025, -0.41933527, -0.14480353, -0.2171271 ,
        -0.03408696],
       [-1.21394141,  0.11198424, -0.41933527, -0.14480353, -0.2171271 ,
        -1.64812038],
       [ 0.97418341,  0.93049361, -0.41933527,  1.87131501, -0.2171271 ,
        -0.03408696]])

In [10]:
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler

numerical_columns_selector = selector(dtype_exclude="category")
categorical_columns_selector = selector(dtype_include="category")

numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", sparse=False)
numerical_preprocessor = StandardScaler()

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

column_preprocessor = ColumnTransformer(
    transformers = [
        ("categories", categorical_preprocessor, categorical_columns),
        ("numerical", numerical_preprocessor, numerical_columns)
    ]
)

logreg_model = LogisticRegression(max_iter=500)

logreg_pipeline = Pipeline(
    steps = [
        ("preprocessing", column_preprocessor),
        ("logistic_regression", logreg_model)
    ]
)

logreg_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('categories',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country']),
                                                 ('numerical', StandardScaler(),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week'])])),
                ('logistic_regression', LogisticRegression(max_iter=500))])

Estimator = ML
Preprocessor = to change/transfrom the data

lin_reg = LinearRegression() # distince -> lin_reg.fit(X,y) -> .fit_predict(X,y) ((y test))


scaler = StandardScaler() -> scaler.fit(X,y) -> .fit_transform(X)


Directed acyclic graph (DAG): орграф, в котором отсутствуют направленные циклы, но могут быть «параллельные» пути, выходящие из одного узла и разными путями приходящие в конечный узел. Направленный ациклический граф является обобщением дерева.

does not look/go back to itself

In [15]:
_=logreg_pipeline.fit(data_train, target_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [16]:
logreg_pipeline[0] # logreg_pipeline['preprocessing']

ColumnTransformer(transformers=[('categories',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country']),
                                ('numerical', StandardScaler(),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week'])])

In [17]:
# to access the values of the estimator

logreg_pipeline[-1].classes_

array(['<=50K', '>50K'], dtype=object)

In [20]:
logreg_pipeline[-1].pedict(data_test)

AttributeError: ignored

In [21]:
logreg_pipeline.predict(data_test)

array(['<=50K', '<=50K', '>50K', ..., '<=50K', '<=50K', '>50K'],
      dtype=object)

In [22]:
logreg_pipeline.score(data_test, target_test) # realisation of the random variable (error metric, it's statistics)

0.8586520350503645

In [25]:
target.value_counts() / len(target)

<=50K    0.760718
>50K     0.239282
Name: class, dtype: float64

### CV for get robust estimate of the error avoid overfitting and choose appropriate metric

In [29]:
from sklearn.model_selection import cross_validate

results = cross_validate(logreg_pipeline,
               data_train,
               target_train,
               scoring = ('accuracy', 'balanced_accuracy', 'recall_weighted', 'f1_weighted', 'neg_log_loss'),
               cv=5,
               return_train_score = True,
               n_jobs=-1
)

In [ ]:
results['']

In [35]:
def get_avg(cv_results):

  result = {}

  for key in results.keys():
    if key.startswith('test_'):
      result[key.replace('test_', '')] = cv_results[key].mean(), cv_results[key].std()
  return result

In [36]:
get_avg(results)

{'accuracy': (0.8499905677604598, 0.0033311751774438127),
 'balanced_accuracy': (0.7637295565177026, 0.0046770703557631995),
 'recall_weighted': (0.8499905677604598, 0.0033311751774438127),
 'f1_weighted': (0.8445412995398229, 0.003448954015535408),
 'neg_log_loss': (-0.322020518737686, 0.005168062598937751)}

In [38]:
from sklearn.model_selection import GridSearchCV

In [43]:
param_grid= {
    'logistic_regression__':(0.8, 0.9, 1.0)
}

grid_search = GridSearchCV(
    logreg_pipeline,
    param_grid=param_grid,
    n_jobs= -1
)

In [44]:
grid_search

GridSearchCV(estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('categories',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         ['workclass',
                                                                          'education',
                                                                          'marital-status',
                                                                          'occupation',
                                                                          'relationship',
                                                                          'race',
                                                                          'sex',
                                                                          'native-country']),
                                                                        ('numerical',
                                                                         StandardScaler(),
                                                                         ['age',
                                                                          'fnlwgt',
                                                                          'education-num',
                                                                          'capital-gain',
                                                                          'capital-loss',
                                                                          'hours-per-week'])])),
                                       ('logistic_regression',
                                        LogisticRegression(max_iter=500))]),
             n_jobs=-1, param_grid={'logistic_regression__': (0.8, 0.9, 1.0)})

In [ ]:
grid_search.fit(data_train, target_train)

In [42]:




from sklearn.model_selection import GridSearchCV

param_grid = {
    'gbtree__learning_rate': (0.01, 0.1, 1, 10),
    'gbtree__max_leaf_nodes': (3, 10, 30)}

model_grid_search = GridSearchCV(
    boosted_pipeline, param_grid=param_grid, n_jobs=-1, cv=5 # inner CV
)

model_grid_search

ValueError: ignored